In [ ]:
import numpy as np
import joblib

# Fungsi untuk memuat model
def load_model(model_file):
    try:
        model = joblib.load(model_file)
        print("Model berhasil dimuat.")
        return model
    except FileNotFoundError:
        print("Model tidak ditemukan.")
        return None
    except AttributeError as e:
        print(f"Error saat memuat model: {e}")
        return None

class DecisionTreeRegressorNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeRegressor:
    def __init__(self, max_depth=5, min_samples_split=3):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._predict(inputs, self.root) for inputs in X])

    def _grow_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        if depth < self.max_depth and num_samples >= self.min_samples_split:
            best_split = self._find_best_split(X, y, num_features)
            if best_split:
                left = self._grow_tree(X[best_split['left_indices']], y[best_split['left_indices']], depth + 1)
                right = self._grow_tree(X[best_split['right_indices']], y[best_split['right_indices']], depth + 1)
                return DecisionTreeRegressorNode(
                    feature_index=best_split['feature_index'],
                    threshold=best_split['threshold'],
                    left=left,
                    right=right
                )
        return DecisionTreeRegressorNode(value=np.mean(y))


    def _predict(self, inputs, node):
        if node.value is not None:
            return node.value
        if inputs[node.feature_index] <= node.threshold:
            return self._predict(inputs, node.left)
        return self._predict(inputs, node.right)



# 3. Implementasi Random Forest Regressor
class RandomForestRegressorFromScratch:
    def __init__(self, n_estimators=60, max_depth=7, min_samples_split=4):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            indices = np.random.choice(X.shape[0], X.shape[0], replace=True)
            X_sample = X[indices]
            y_sample = y[indices]
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(predictions, axis=0)

# Fungsi untuk meminta input dari pengguna
def get_user_input():
    building_size = np.log1p(float(input("Luas bangunan (m²): ")))
    land_size = np.log1p(float(input("Luas tanah (m²): ")))
    electricity = np.log1p(float(input("Daya listrik (Watt): ")))
    year_built = np.log1p(float(input("Tahun dibangun: ")))
    bathrooms = np.log1p(float(input("Jumlah kamar mandi: ")))
    bedrooms = np.log1p(float(input("Jumlah kamar tidur: ")))
    carports = np.log1p(float(input("Jumlah carport: ")))
    garages = np.log1p(float(input("Jumlah garasi: ")))
    floors = np.log1p(float(input("Jumlah lantai: ")))

    city = input("Lokasi kota (Bogor, Depok, Jakarta, Tangerang, Bekasi): ").strip()
    city_features = ["Bogor", "Depok", "Jakarta Barat", "Jakarta Pusat", "Jakarta Selatan", "Jakarta Timur", "Jakarta Utara", "Tangerang", "Bekasi"]
    city_input = [1 if city == c else 0 for c in city_features]

    inputs = np.array([[building_size, land_size, electricity, year_built,
                        bathrooms, bedrooms, carports, garages, floors] + city_input])

    return inputs

# Fungsi untuk melakukan prediksi
def predict_price(model, input_features):
    predicted_log_price = model.predict(input_features)
    predicted_price = np.expm1(predicted_log_price)  # Mengkonversi kembali dari log
    return predicted_price[0]

# Fungsi utama
def main():
    MODEL_FILE = "/content/random_forest_model.joblib"  # Ubah sesuai nama file model Anda

    model = load_model(MODEL_FILE)
    if model is None:
        return

    user_input = get_user_input()

    predicted_price = predict_price(model, user_input)
    print(f"Prediksi harga rumah: Rp {predicted_price:,.2f}")

if __name__ == "__main__":
    main()